In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from keras.models import load_model

In [ ]:
x=pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
len(x)

In [ ]:
x.head()

In [ ]:
fractions=np.array([0.9,0.1])
x=x.sample(frac=1)
train_df,test_df=np.array_split(x,(fractions[:-1].cumsum()*len(x)).astype(int))

In [ ]:
train_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(train_df['image_id'], train_df['label'],
                                                    stratify=train_df['label'], 
                                                    test_size=0.3)

In [ ]:
X_train=pd.DataFrame(X_train)
X_validate=pd.DataFrame(X_validate)
y_train=pd.DataFrame(y_train)
y_validate=pd.DataFrame(y_validate)

In [ ]:
X_validate['label']=y_validate

In [ ]:
X_validate.head()

In [ ]:
X_train['label']=y_train

In [ ]:
X_train.head()

In [ ]:
y=pd.get_dummies(X_train['label'])
y
X_train=X_train.drop('label',axis=1)
train_df=pd.concat([X_train,y],axis=1)
train_df.columns=['image_id','Cassava Bacterial Blight (CBB)','Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']

train_df = train_df.astype({"image_id": object, "Cassava Bacterial Blight (CBB)": np.float32,"Cassava Brown Streak Disease (CBSD)": np.float32,"Cassava Green Mottle (CGM)": np.float32,"Cassava Mosaic Disease (CMD)": np.float32,"Healthy": np.float32})





In [ ]:
y=pd.get_dummies(X_validate['label'])
y
X_validate=X_validate.drop('label',axis=1)
validate_df=pd.concat([X_validate,y],axis=1)
validate_df.columns=['image_id','Cassava Bacterial Blight (CBB)','Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']

validate_df = validate_df.astype({"image_id": object, "Cassava Bacterial Blight (CBB)": np.float32,"Cassava Brown Streak Disease (CBSD)": np.float32,"Cassava Green Mottle (CGM)": np.float32,"Cassava Mosaic Disease (CMD)": np.float32,"Healthy": np.float32})





In [ ]:
y=pd.get_dummies(test_df['label'])
y
test_df=test_df.drop('label',axis=1)
test_dff=pd.concat([test_df,y],axis=1)
test_dff.columns=['image_id','Cassava Bacterial Blight (CBB)','Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']

test_dff = test_dff.astype({"image_id": object, "Cassava Bacterial Blight (CBB)": np.float32,"Cassava Brown Streak Disease (CBSD)": np.float32,"Cassava Green Mottle (CGM)": np.float32,"Cassava Mosaic Disease (CMD)": np.float32,"Healthy": np.float32})





In [ ]:
print(len(train_df),len(validate_df),len(test_dff))

In [ ]:
labels = [
          'Cassava Bacterial Blight (CBB)',
          'Cassava Brown Streak Disease (CBSD)', 'Cassava Green Mottle (CGM)',
          'Cassava Mosaic Disease (CMD)',
          'Healthy'
]

In [ ]:
def data_leakage_check(df1,df2,col):
    
    df1_unique=set(df1[col])
    df2_unique=set(df2[col])
    
    same_images_in_both_groups=list(df1_unique.intersection(df2_unique))
    
    leakage=len(same_images_in_both_groups)!=0
    
    return leakage

In [ ]:
print("Leakage between train and valid",data_leakage_check(train_df,validate_df,'image_id'))
print("Leakage between train and test",data_leakage_check(train_df,test_dff,'image_id'))
print("Leakage between test and valid",data_leakage_check(test_dff,validate_df,'image_id'))

In [ ]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=32, target_w = 320, target_h = 320):
    """
    Return generator for training set, normalizing using batch
    statistics.

    Args:
      train_df (dataframe): dataframe specifying training data.
      image_dir (str): directory where image files are held.
      x_col (str): name of column in df that holds filenames.
      y_cols (list): list of strings that hold y labels for images.
      sample_size (int): size of sample to use for normalization statistics.
      batch_size (int): images per batch to be fed into model during training.
      seed (int): random seed.
      target_w (int): final width of input images.
      target_h (int): final height of input images.
    
    Returns:
        train_generator (DataFrameIterator): iterator over training set
    """        
    print("getting train generator...") 
    # normalize images
    image_generator = ImageDataGenerator(
        rotation_range=90,
        fill_mode='nearest',
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.5,1.5],
        samplewise_center=True,
        samplewise_std_normalization= True)
    
    # flow from directory with specified batch size
    # and target image size
    generator = image_generator.flow_from_dataframe(
            dataframe=df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=shuffle,
            
            target_size=(target_w,target_h))
    
    return generator

In [ ]:
def get_test_and_valid_generator(valid_df, test_df, train_df, image_dir, x_col, y_cols, sample_size=200, batch_size=32, target_w = 320, target_h = 320):
    """
    Return generator for validation set and test test set using 
    normalization statistics from training set.

    Args:
      valid_df (dataframe): dataframe specifying validation data.
      test_df (dataframe): dataframe specifying test data.
      train_df (dataframe): dataframe specifying training data.
      image_dir (str): directory where image files are held.
      x_col (str): name of column in df that holds filenames.
      y_cols (list): list of strings that hold y labels for images.
      sample_size (int): size of sample to use for normalization statistics.
      batch_size (int): images per batch to be fed into model during training.
      seed (int): random seed.
      target_w (int): final width of input images.
      target_h (int): final height of input images.
    
    Returns:
        test_generator (DataFrameIterator) and valid_generator: iterators over test set and validation set respectively
    """
    print("getting train and valid generators...")
    # get generator to sample dataset
    raw_train_generator = ImageDataGenerator().flow_from_dataframe(
        dataframe=train_df, 
        directory=IMAGE_DIR, 
        x_col="image_id", 
        y_col=labels, 
        class_mode="raw", 
        batch_size=sample_size, 
        shuffle=True, 
        target_size=(target_w, target_h))
    
    # get data sample
    batch = raw_train_generator.next()
    data_sample = batch[0]

    # use sample to fit mean and std for test set generator
    image_generator = ImageDataGenerator(
        
        featurewise_center=True,
        featurewise_std_normalization= True)
    
    # fit generator to sample from training data
    image_generator.fit(data_sample)

    # get test generator
    valid_generator = image_generator.flow_from_dataframe(
            dataframe=validate_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            
            target_size=(target_w,target_h))

    test_generator = image_generator.flow_from_dataframe(
            dataframe=test_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            
            target_size=(target_w,target_h))
    return valid_generator, test_generator

In [ ]:
IMAGE_DIR='../input/cassava-leaf-disease-classification/train_images'
train_generator=get_train_generator(train_df,IMAGE_DIR,"image_id",labels)
valid_generator,test_generator=get_test_and_valid_generator(validate_df,test_dff,train_df,IMAGE_DIR,"image_id",labels)

In [ ]:
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_generator.labels, axis=0),color='cadetblue')
plt.ylabel("Frequency")
plt.show()

In [ ]:
def compute_class_freqs(labels):
    
    N=len(labels)
    
    pos_f=np.sum(labels,axis=0)/N
    neg_f=1-np.sum(labels,axis=0)/N
    
    return pos_f,neg_f

In [ ]:
f_pos,f_neg=compute_class_freqs(train_generator.labels)
f_pos

In [ ]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": f_pos})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} for l,v in enumerate(f_neg)], ignore_index=True)
plt.xticks(rotation=90)
f = sns.barplot(x="Class", y="Value", hue="Label" ,data=data,palette=['#5f9ea0','#87CEEB'])

In [ ]:
pos_weights=f_neg
neg_weights=f_pos
pos_c=f_pos*pos_weights
neg_c=f_neg*neg_weights

In [ ]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": pos_c})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} 
                        for l,v in enumerate(neg_c)], ignore_index=True)
plt.xticks(rotation=90)
sns.barplot(x="Class", y="Value", hue="Label" ,data=data,palette=['#5f9ea0','#87CEEB']);

In [ ]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    """
    Return weighted loss function given negative weights and positive weights.

    Args:
      pos_weights (np.array): array of positive weights for each class, size (num_classes)
      neg_weights (np.array): array of negative weights for each class, size (num_classes)
    
    Returns:
      weighted_loss (function): weighted loss function
    """
    def weighted_loss(y_true, y_pred):
        """
        Return weighted loss value. 

        Args:
            y_true (Tensor): Tensor of true labels, size is (num_examples, num_classes)
            y_pred (Tensor): Tensor of predicted labels, size is (num_examples, num_classes)
        Returns:
            loss (float): overall scalar loss summed across all classes
        """
        # initialize loss to zero
        loss = 0.0
        
        ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

        for i in range(len(pos_weights)):
            # for each class, add average weighted loss for that class 
            loss += -(K.mean((pos_weights[i] * y_true[:,i] * K.log(y_pred[:,i] + epsilon)) + (neg_weights[i] * (1-y_true[:,i]) * K.log(1-y_pred[:,i] + epsilon)),axis = 0)) #complete this line
        return loss
    
        ### END CODE HERE ###
    return weighted_loss

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

from keras.models import load_model





In [ ]:
def tensor_to_array(array_value):
    return array_value.numpy()

In [ ]:
# Test
import tensorflow as tf
sess = tf.compat.v1.keras.backend.get_session()
with sess.as_default() as sess:
    print("Test example:\n")
    y_true = K.constant(np.array(
        [[1, 1, 1],
         [1, 1, 0],
         [0, 1, 0],
         [1, 0, 1]]
    ))
    print("y_true:\n")
    print(tensor_to_array(y_true))

    w_p = np.array([0.25, 0.25, 0.5])
    w_n = np.array([0.75, 0.75, 0.5])
    print("\nw_p:\n")
    print(w_p)

    print("\nw_n:\n")
    print(w_n)

    y_pred_1 = K.constant(0.7*np.ones(y_true.shape))
    print("\ny_pred_1:\n")
    print(tensor_to_array(y_pred_1))

    y_pred_2 = K.constant(0.3*np.ones(y_true.shape))
    print("\ny_pred_2:\n")
    print(tensor_to_array(y_pred_2))

    # test with a large epsilon in order to catch errors
    L = get_weighted_loss(w_p, w_n, epsilon=1)

    print("\nIf we weighted them correctly, we expect the two losses to be the same.")
    L1 = tensor_to_array(L(y_true, y_pred_1))
    L2 = tensor_to_array(L(y_true, y_pred_2))
    
    
    print(f"\nL(y_pred_1)= {L1:.4f}, L(y_pred_2)= {L2:.4f}")
    print(f"Difference is L1 - L2 = {L1 - L2:.4f}")

In [ ]:
import keras.backend as K

In [ ]:
def sensitivity(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

In [ ]:
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [ ]:
from tensorflow.keras.applications import DenseNet169
base_model = DenseNet169(weights='imagenet', include_top=False,input_shape=(320,320,3))


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
import tensorflow as tf
from keras.layers import BatchNormalization,Dropout
from keras.callbacks import ReduceLROnPlateau
import keras	
x=base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)

predictions=Dense(5,activation='sigmoid')(x)
import keras
model=Model(inputs=base_model.input,outputs=predictions)
optimiser=tf.keras.optimizers.Adam(lr=0.0001)


es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights = True)

model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001),loss=get_weighted_loss(pos_weights,neg_weights),metrics=['accuracy',specificity,sensitivity])


In [ ]:
import numpy as np

In [ ]:
history = model.fit(train_generator, 
                              validation_data=valid_generator,
                              steps_per_epoch=int( np.ceil(len(train_df)/ 32) ),
                    validation_steps=int( np.ceil(len(validate_df)/ 32) ),
                              epochs = 90,
                              callbacks=[es]
                    
                    
                    
                   )


In [ ]:
predicted_vals = model.predict_generator(test_generator, steps = len(test_generator))

In [ ]:
from shutil import copyfile


In [ ]:
import random
tf.compat.v1.enable_eager_execution()
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K
from keras.preprocessing import image
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.compat.v1.logging import INFO, set_verbosity

random.seed(a=None, version=2)

set_verbosity(INFO)


def get_mean_std_per_batch(image_path, df, H=256, W=256):
    sample_data = []
    for idx, img in enumerate(df.sample(100)["image_id"].values):
        # path = image_dir + img
        sample_data.append(
            np.array(image.load_img(image_path, target_size=(H, W))))

    mean = np.mean(sample_data[0])
    std = np.std(sample_data[0])
    return mean, std


def load_image(img, image_dir, df, preprocess=True, H=256, W=256):
    """Load and preprocess image."""
    img_path = image_dir + img
    mean, std = get_mean_std_per_batch(img_path, df, H=H, W=W)
    x = image.load_img(img_path, target_size=(H, W))
    if preprocess:
        x -= mean
        x /= std
        x = np.expand_dims(x, axis=0)
    return x

from tensorflow import GradientTape
def grad_cam(input_model, image, cls, layer_name, H=256, W=256):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]

    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (W, H), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam


def compute_gradcam(model, img, image_dir, df, labels, selected_labels,
                    layer_name='bn'):
    preprocessed_input = load_image(img, image_dir, df)
    predictions = model.predict(preprocessed_input)

    print("Loading original image")
    plt.figure(figsize=(15, 10))
    plt.subplot(151)
    plt.title("Original")
    plt.axis('off')
    plt.imshow(load_image(img, image_dir, df, preprocess=False), cmap='gray')

    j = 1
    for i in range(len(labels)):
        if labels[i] in selected_labels:
            print(f"Generating gradcam for class {labels[i]}")
            gradcam = grad_cam(model, preprocessed_input, i, layer_name)
            plt.subplot(151 + j)
            plt.title(f"{labels[i]}: p={predictions[0][i]:.3f}")
            plt.axis('off')
            plt.imshow(load_image(img, image_dir, df, preprocess=False),
                       cmap='gray')
            plt.imshow(gradcam, cmap='jet', alpha=min(0.5, predictions[0][i]))
            j += 1


def get_roc_curve(labels, predicted_vals, generator):
    auc_roc_vals = []
    for i in range(len(labels)):
        try:
            gt = generator.labels[:, i]
            pred = predicted_vals[:, i]
            auc_roc = roc_auc_score(gt, pred)
            auc_roc_vals.append(auc_roc)
            fpr_rf, tpr_rf, _ = roc_curve(gt, pred)
            plt.figure(1, figsize=(10, 10))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.plot(fpr_rf, tpr_rf,
                     label=labels[i] + " (" + str(round(auc_roc, 3)) + ")")
            plt.xlabel('False positive rate')
            plt.ylabel('True positive rate')
            plt.title('ROC curve')
            plt.legend(loc='best')
        except:
            print(
                f"Error in generating ROC curve for {labels[i]}. "
                f"Dataset lacks enough examples."
            )
    plt.show()
    return auc_roc_vals

In [ ]:
auc_rocs =get_roc_curve(labels, predicted_vals, test_generator)

In [ ]:
plt.plot(history.history['loss'],color='indianred')
plt.plot(history.history['val_loss'],color='goldenrod')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['sensitivity'],color='indianred')
plt.plot(history.history['val_sensitivity'],color='goldenrod')
plt.ylabel('Sensitivity')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['specificity'],color='indianred')
plt.plot(history.history['val_specificity'],color='goldenrod')
plt.ylabel('Specificity')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
sensitivity(test_generator.labels,predicted_vals)

In [ ]:
specificity(test_generator.labels,predicted_vals)